In [25]:
import pandas as pd
import numpy as np

# Load data and unify column names
df = pd.read_csv(r"C:\My stuff\Coding\ML project\KiranveerSingh_projectfinal\Notebook\Model_robusting\HDFCBANK.csv")
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

# Parse date
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Standardize price columns
col_map = {
    'Open': 'Open',
    'High': 'High',
    'Low': 'Low',
    'Last_Traded_Price': 'Close',  # Adjust if 'Close' is directly available
    'Close': 'Close',  # Prefer 'Close' if present
    'Share_Volume': 'Volume',
    'Symbol': 'Symbol',
    'Series': 'Series'
}
# If both 'Close' and 'Last_Traded_Price' present, prefer 'Close'
if 'Close' in df.columns:
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
elif 'Last_Traded_Price' in df.columns:
    df['Close'] = pd.to_numeric(df['Last_Traded_Price'], errors='coerce')

# Convert numerics
for col in ['Open', 'High', 'Low', 'Volume']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop duplicates & invalid data
df = df.drop_duplicates(subset=['Symbol', 'Date'])
df = df.dropna(subset=['Open', 'High', 'Low', 'Close', 'Volume'])
for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    df = df[df[col] > 0]

# Sort by date for each symbol
df = df.sort_values(['Symbol', 'Date']).reset_index(drop=True)


In [26]:
# Create target: Will_Grow == 1 if next day's close > today's, else 0
df['Next_Close'] = df.groupby('Symbol')['Close'].shift(-1)
df['Will_Grow'] = (df['Next_Close'] > df['Close']).astype(int)
df = df.drop(columns=['Next_Close'])

# Remove last row per symbol with undefined target
df = df.groupby('Symbol', group_keys=False).apply(lambda g: g.iloc[:-1]).reset_index(drop=True)


C:\Users\mrran\AppData\Local\Temp\ipykernel_23288\2388469121.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Symbol', group_keys=False).apply(lambda g: g.iloc[:-1]).reset_index(drop=True)


In [28]:
# Moving averages
for window in [5, 10, 20]:
    df[f"SMA_{window}"] = df.groupby("Symbol")["Close"].transform(lambda x: x.rolling(window).mean())

# Daily returns
df['Return_1D'] = df.groupby('Symbol')['Close'].pct_change()

# Volatility
df['Volatility_10'] = df.groupby('Symbol')['Return_1D'].transform(lambda x: x.rolling(10).std())

# Price ratios
df['High_Low_Ratio'] = df['High'] / df['Low']
df['Open_Close_Ratio'] = df['Open'] / df['Close']

# Volume spike vs. 10-day rolling mean
df['Volume_SMA_10'] = df.groupby('Symbol')['Volume'].transform(lambda x: x.rolling(10).mean())
df['Volume_Ratio'] = df['Volume'] / df['Volume_SMA_10']

# Lagged close and volume
for lag in [1, 2, 3]:
    df[f'Close_Lag_{lag}'] = df.groupby('Symbol')['Close'].shift(lag)
    df[f'Volume_Lag_{lag}'] = df.groupby('Symbol')['Volume'].shift(lag)

# RSI Calculation
def calc_rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=period-1, adjust=False).mean()
    ema_down = down.ewm(com=period-1, adjust=False).mean()
    rs = ema_up / ema_down
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI_14'] = df.groupby("Symbol")["Close"].transform(lambda x: calc_rsi(x, 14))

# Drop rows with NaN caused by rolling/lags
df = df.dropna().reset_index(drop=True)

# Save the processed data
df.to_csv("cleaned_featured_nifty500.csv", index=False)

print("Saved cleaned and feature-engineered data to 'cleaned_featured_HDFCBANK.csv'")
print(df.head())


Saved cleaned and feature-engineered data to 'cleaned_featured_HDFCBANK.csv'
        Date    Symbol Series  Prev_Close    Open     High      Low     Last  \
0 2011-06-28  HDFCBANK     EQ     2416.15  2429.4  2430.00  2404.55  2424.65   
1 2011-06-29  HDFCBANK     EQ     2423.05  2435.0  2499.00  2435.00  2495.00   
2 2011-06-30  HDFCBANK     EQ     2489.10  2499.0  2524.95  2476.15  2503.50   
3 2011-07-01  HDFCBANK     EQ     2515.55  2540.0  2547.60  2478.00  2501.35   
4 2011-07-04  HDFCBANK     EQ     2501.25  2535.0  2542.00  2512.40  2536.00   

     Close     VWAP  ...  Open_Close_Ratio  Volume_SMA_10  Volume_Ratio  \
0  2423.05  2419.64  ...          1.002621       631161.4      0.965967   
1  2489.10  2467.29  ...          0.978265       663296.4      1.545680   
2  2515.55  2506.02  ...          0.993421       755375.9      1.776678   
3  2501.25  2497.60  ...          1.015492       750832.7      0.886006   
4  2532.20  2528.72  ...          1.001106       744321.0      0.96